# ***AI Shield: Integrated Distress Monitoring and Alert System***

# **CODE**

In [ ]:
!pip install kagglehub
!pip install --upgrade tensorflow

import kagglehub

  Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.18.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.3 MB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
# Download the dataset
path = kagglehub.dataset_download("aananehsansiam/audio-dataset-of-scream-and-non-scream")

print("Path to dataset files:", path)

import os
print(os.listdir(path))

100%|██████████| 333M/333M [00:04<00:00, 74.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/aananehsansiam/audio-dataset-of-scream-and-non-scream/versions/1
['Converted_Separately']


In [ ]:
!kaggle datasets download -d aananehsansiam/audio-dataset-of-scream-and-non-scream
!unzip audio-dataset-of-scream-and-non-scream.zip


Dataset URL: https://www.kaggle.com/datasets/aananehsansiam/audio-dataset-of-scream-and-non-scream
License(s): unknown
 93% 310M/333M [00:01<00:00, 236MB/s]
100% 333M/333M [00:01<00:00, 216MB/s]
Archive:  audio-dataset-of-scream-and-non-scream.zip
  inflating: Converted_Separately/non_scream/1.wav  
  inflating: Converted_Separately/non_scream/10.wav  
  inflating: Converted_Separately/non_scream/100.wav  
  inflating: Converted_Separately/non_scream/1000.wav  
  inflating: Converted_Separately/non_scream/1001.wav  
  inflating: Converted_Separately/non_scream/1002.wav  
  inflating: Converted_Separately/non_scream/1003.wav  
  inflating: Converted_Separately/non_scream/1004.wav  
  inflating: Converted_Separately/non_scream/1005.wav  
  inflating: Converted_Separately/non_scream/1006.wav  
  inflating: Converted_Separately/non_scream/1007.wav  
  inflating: Converted_Separately/non_scream/1008.wav  
  inflating: Converted_Separately/non_scream/1009.wav  
  inflating: Converted_Separat

In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [ ]:
# Step 1: Load and Inspect Dataset
dataset_path = "/root/.cache/kagglehub/datasets/aananehsansiam/audio-dataset-of-scream-and-non-scream/versions/1"  # Update with actual path
audio_files = []
labels = []

# Walk through dataset directory
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".wav"):  # Assuming the audio files are in .wav format
            file_path = os.path.join(root, file)
            label = os.path.basename(root)  # Use the parent folder as label
            audio_files.append(file_path)
            labels.append(label)

print(f"Loaded {len(audio_files)} audio files.")


Loaded 3128 audio files.


In [ ]:
# Step 2: Extract Features
def extract_features(file_path, sr=22050):
    try:
        # Load audio file
        audio, sample_rate = librosa.load(file_path, sr=sr)
        # Extract MFCC features
        mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        # Return the mean of MFCCs (flattened array)
        return np.mean(mfcc.T, axis=0)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Extract features for all audio files
features = []
for file in audio_files:
    feature = extract_features(file)
    if feature is not None:
        features.append(feature)

# Convert to NumPy arrays
X = np.array(features)
y = np.array(labels)


In [ ]:
# Step 3: Encode Labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)


In [ ]:
# Step 4: Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

In [ ]:
# Step 5: Build and Train the Model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Convert one-hot encoded labels to binary
y_train_binary = np.argmax(y_train, axis=1)
y_test_binary = np.argmax(y_test, axis=1)

# Train the model
history = model.fit(X_train, y_train_binary, validation_data=(X_test, y_test_binary), epochs=100, batch_size=32)


Epoch 1/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.6328 - loss: 6.4946 - val_accuracy: 0.8658 - val_loss: 0.6066
Epoch 2/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7936 - loss: 1.1811 - val_accuracy: 0.8371 - val_loss: 0.3665
Epoch 3/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7977 - loss: 0.7023 - val_accuracy: 0.8738 - val_loss: 0.3734
Epoch 4/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8353 - loss: 0.4406 - val_accuracy: 0.8834 - val_loss: 0.3609
Epoch 5/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8532 - loss: 0.4003 - val_accuracy: 0.9042 - val_loss: 0.3264
Epoch 6/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8535 - loss: 0.3614 - val_accuracy: 0.8770 - val_loss: 0.3226
Epoch 7/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8602 - loss: 0.3481 - val_accuracy: 0.8930 - val_loss: 0.2910
Epoch 8/20
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8718 - loss: 0.3082 - val_accuracy: 0.8882 - val_loss

In [ ]:
# Step 6: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test_binary)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Custom Threshold Prediction
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  # For custom threshold (0.5 is default)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9082 - loss: 0.2465 
Test Accuracy: 90.10%
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [ ]:
# Step 7: Function to Predict a Single Audio File
def predict_audio(file_path, model, label_encoder, sr=22050):
    # Extract features from the audio file
    feature = extract_features(file_path, sr=sr)
    if feature is None:
        print(f"Error: Could not process the file {file_path}")
        return None

    # Reshape the feature array for prediction
    feature = np.expand_dims(feature, axis=0)

    # Get model prediction
    prediction = model.predict(feature)

    # Decode the predicted class
    predicted_class = (prediction > 0.5).astype(int)  # Modified for binary classification
    label = label_encoder.inverse_transform(predicted_class.flatten())  # Modified decoding

    return label[0], prediction[0]

In [ ]:
# Step 8: Test with a Sample Audio File
sample_audio_path = "/content/273153__phantastonia__child-scream2.wav"  # Replace with the actual path to your test audio file
predicted_label, prediction_probabilities = predict_audio(sample_audio_path, model, label_encoder)

# Display the results
print(f"Predicted Label: {predicted_label}")
print(f"Prediction Probabilities: {prediction_probabilities}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
Predicted Label: scream
Prediction Probabilities: [0.960439]


In [ ]:
!pip install twilio

from geopy.geocoders import Nominatim
from twilio.rest import Client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.4 MB/s eta 0:00:00


In [ ]:
# Updated get_gps_coordinates function
def get_gps_coordinates(address=None):
    """
    Retrieves GPS coordinates.
    If an address is provided, it fetches the coordinates for that address.
    If no address is provided, it uses a default address for testing.
    """
    try:
        # Use geopy to get coordinates dynamically
        geolocator = Nominatim(user_agent="geoapi")

        if address:
            location = geolocator.geocode(address)
        else:
            # Default address if none is provided
            location = geolocator.geocode("Amrita Vishwa Vidyapeetham, Amritapuri, Karunagapally, Kerala, India")

        if location:
            latitude = location.latitude
            longitude = location.longitude
            print(f"GPS Coordinates: {latitude}, {longitude}")
            return latitude, longitude
        else:
            print("Error: Unable to retrieve GPS coordinates.")
            return None, None
    except Exception as e:
        print(f"GPS retrieval failed: {e}")
        return None, None

def preprocess_and_predict(audio_path, model, label_encoder):
    try:
        feature = extract_features(audio_path)
        feature = np.expand_dims(feature, axis=0)
        prediction = model.predict(feature)
        scream_detected = prediction[0][0] > 0.7
        return scream_detected, prediction[0]
    except Exception as e:
        print(f"Error during prediction: {e}")
        return False, None

# Function to send SMS alert
def send_sms_alert(gps_coordinates, twilio_sid, twilio_token, twilio_from_number, twilio_to_number):
    client = Client(twilio_sid, twilio_token)
    message = client.messages.create(
        body=f"Scream detected! Location: {gps_coordinates}",
        from_=twilio_from_number,
        to=twilio_to_number
    )
    print(f"Alert sent: {message.sid}")

def distress_detection_pipeline(audio_path, model, label_encoder, twilio_sid, twilio_token, twilio_from_number, twilio_to_number, address=None):
    """
    Main pipeline for distress detection.
    If a scream is detected, fetch GPS coordinates (optionally using an address)
    and send an SMS alert with the coordinates.
    """
    scream_detected, prediction_probabilities = preprocess_and_predict(audio_path, model, label_encoder)

    if scream_detected:
        latitude, longitude = get_gps_coordinates(address)
        if latitude and longitude:
            gps_coordinates = f"{latitude}, {longitude}"
            send_sms_alert(gps_coordinates, twilio_sid, twilio_token, twilio_from_number, twilio_to_number)
        else:
            print("GPS coordinates not available.")
    else:
        print("No scream detected.")

In [ ]:
# Run the pipeline
test_audio_path = "/content/273153__phantastonia__child-scream2.wav"
twilio_sid = ""  # Replace with your Twilio SID
twilio_token = ""  # Replace with your Twilio Auth Token
twilio_from_number = ""  # Replace with your Twilio phone number
twilio_to_number = ""  # Replace with the trusted contact's phone number
distress_detection_pipeline(test_audio_path, model, label_encoder, twilio_sid, twilio_token, twilio_from_number, twilio_to_number)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
GPS Coordinates: 9.09633245, 76.49003274438223
Alert sent: SMaec95499799fd79931a36b4321392847
